##Проект 1799: "Разработка системы предсказания параметров цифровых схем с использованием методов машинного обучения"

##Подготовка и исследование данных

###Импорт всех библиотек

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn, optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import os
import math



sns.set(font_scale=1.4, style='whitegrid')

##Инициализация устройства

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

##Загрузка данных

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data = pd.read_csv('drive/MyDrive/embeddings.csv') #Файл содержит уже очищенные от выбросов данные
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41192 entries, 0 to 41191
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      41192 non-null  int64  
 1   File            41192 non-null  object 
 2   Area            41192 non-null  float64
 3   Delay           41192 non-null  float64
 4   embedding       41192 non-null  object 
 5   New_embeddings  41192 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.9+ MB


In [7]:
data.drop(['New_embeddings', 'Unnamed: 0'], axis=1, inplace=True)
data

,File,Area,Delay,embedding
0,CCGRCG1,11.26,139.61,"[[-0.0021449089981615543, 0.000945725420024246..."
1,CCGRCG2,21.27,147.38,"[[-0.0021449089981615543, 0.000945725420024246..."
2,CCGRCG3,11.25,194.25,"[[-0.002141991164535284, 0.0009532237891107798..."
3,CCGRCG4,11.25,107.18,"[[-0.0021485977340489626, 0.000949225854128599..."
4,CCGRCG5,22.50,160.62,"[[-0.0021652388386428356, 0.000933241331949830..."
...,...,...,...,...
41187,CCGRCG11005,60.06,265.41,"[[-0.0021583845373243093, 0.000951010966673493..."
41188,CCGRCG11006,102.57,358.25,"[[-0.00213915528729558, 0.0009182162466458976,..."
41189,CCGRCG11007,140.06,459.67,"[[-0.0021358621306717396, 0.000899522274266928..."
41190,CCGRCG11008,401.49,780.42,"[[-0.002138545038178563, 0.0009022692102007568..."


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41192 entries, 0 to 41191
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   File       41192 non-null  object 
 1   Area       41192 non-null  float64
 2   Delay      41192 non-null  float64
 3   embedding  41192 non-null  object 
dtypes: float64(2), object(2)
memory usage: 1.3+ MB


Посмотрим на наши эмбеддинги

In [9]:
data['embedding'][255]

'[[-0.002152559347450733, 0.0009587233071215451, 0.020421821624040604, 0.03604637086391449, -0.03722260519862175, -0.02848442643880844, 0.025840260088443756, 0.03590499237179756, -0.02006523311138153, -0.01506724115461111, 0.02954143099486828, -0.006135474890470505, -0.018150070682168007, 0.02622663415968418, -0.01944422908127308, -0.007272362243384123, 0.01151384599506855, 0.003970533609390259, -0.03316284343600273, -0.037806760519742966, 0.029262060299515724, 0.020292656496167183, 0.02704456076025963, 0.0030577874276787043, 0.025405921041965485], [-0.013612494803965092, -0.0037808783818036318, 0.023076921701431274, -0.03008740209043026, -0.01574854366481304, -0.030041908845305443, -0.003714316990226507, 0.03815561160445213, -0.02927679568529129, -0.009338419884443283, -0.0077545857056975365, 0.03230588510632515, -0.023739874362945557, 0.0001809560926631093, -0.01901940628886223, -0.03841422498226166, 0.02003874070942402, -0.03503160551190376, -0.017578089609742165, -0.000128517043776

Эмбеддинги представлены в виде строк, что не подходит для обучения моделей. Исправим это и создадим массивы чисел

In [10]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [11]:
new_embedding = []
for str in data['embedding']:
    result = []
    if '\n' in str:
        res = str.split('\n')
    else:
        res = str.split('], ')
    for mass in res:
        s = mass.replace(', dtype=float32',' ').replace('(', ' ').replace(')',' ').replace(',',' ').replace('[','').replace(']','').split(' ')
        tmp = []
        for word in s:
            if is_number(word):
                tmp.append(float(word))
        result.append(tmp)
    r = [np.array(sublist) for sublist in result]
    new_embedding.append(r)

Сравним длины полученных массивов

In [12]:
len(new_embedding[1]), len(new_embedding[2435])

(8, 38)

Так как np.array и torch.FloatTensor не могут иметь массивы разных размеров, дополним их все нулевыми эмбеддингами до максимальной длины самой большой последовательности

In [13]:
max_subarrays = max(len(arr) for arr in new_embedding)
print(max_subarrays)
print(min(len(arr) for arr in new_embedding))

for i in range(len(new_embedding)):
    while len(new_embedding[i]) < max_subarrays:
        new_embedding[i] = np.vstack([new_embedding[i], np.zeros(25)])

141
4


In [14]:
X = np.array(new_embedding)
y_area = np.array(data['Area'])
y_delay = np.array(data['Delay'])

In [ ]:
X.shape

(41192, 141, 25)

Создадим отдельно данные для обучения/тестирования моделей предсказания площади и моделей предсказания задержки

In [15]:
# Для площади
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X, y_area, test_size=0.2, random_state=42)
# Для задержки
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, y_delay, test_size=0.2, random_state=42)

Переведём всё в тензоры, так как модели реализованы на при помощи PyTorch

In [27]:
X_train_a = torch.FloatTensor(np.array([torch.FloatTensor(X) for X in X_train_a]))
X_test_a = torch.FloatTensor(np.array([torch.FloatTensor(X) for X in X_test_a]))
y_train_a = torch.FloatTensor(y_train_a)
y_test_a = torch.FloatTensor(y_test_a)

In [28]:
X_train_d = torch.FloatTensor(np.array([torch.FloatTensor(X) for X in X_train_d]))
X_test_d = torch.FloatTensor(np.array([torch.FloatTensor(X) for X in X_test_d]))
y_train_d = torch.FloatTensor(y_train_d)
y_test_d = torch.FloatTensor(y_test_d)

In [ ]:
X_train_a.shape, y_train_a.shape

(torch.Size([32953, 141, 24]), torch.Size([32953]))

In [ ]:
# train = dict(zip(y_train, X_train))
# test = dict(zip(y_test, X_test))

##Реализация функций и классов для создания и обучения моделей

In [18]:
# Функция для разбиения данных на батчи
def get_dataloaders(trainset, testset, batch_size=4):

  trainloader_X = torch.utils.data.DataLoader(trainset[0], batch_size=batch_size,
                                            shuffle=False, num_workers=2)
  trainloader_y = torch.utils.data.DataLoader(trainset[1], batch_size=batch_size,
                                              shuffle=False, num_workers=2)

  testloader_X = torch.utils.data.DataLoader(testset[0], batch_size=batch_size,
                                           shuffle=False, num_workers=2)
  testloader_y = torch.utils.data.DataLoader(testset[1], batch_size=batch_size,
                                           shuffle=False, num_workers=2)

  return [trainloader_X, trainloader_y], [testloader_X, testloader_y]
  # return trainloader, testloader

In [19]:
# Функция для отображения графика обучения
def plot_training(train_losses, valid_losses, valid_accuracies):
  plt.figure(figsize=(12, 9))
  plt.subplot(2, 1, 1)
  plt.xlabel('epoch')
  plt.plot(train_losses[1:], label='train_loss')
  plt.plot(valid_losses[1:], label='valid_loss')
  plt.legend()

  plt.subplot(2, 1, 2)
  plt.xlabel('epoch')
  plt.plot(valid_accuracies, label='Mean Realative Error')
  plt.legend()

Реализуем необходимый нам модуль слоя внимания

In [20]:
class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hid_dim, n_heads, dropout, device):
    super().__init__()

    assert hid_dim % n_heads == 0

    self.hid_dim = hid_dim
    self.n_heads = n_heads
    self.head_dim = hid_dim // n_heads

    self.fc_q = nn.Linear(hid_dim, hid_dim)
    self.fc_k = nn.Linear(hid_dim, hid_dim)
    self.fc_v = nn.Linear(hid_dim, hid_dim)

    self.fc_o = nn.Linear(hid_dim, hid_dim)

    self.dropout = nn.Dropout(dropout)

    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self, query, key, value, mask=None):
    batch_size = query.shape[0]

    Q = self.fc_q(query)
    K = self.fc_k(key)
    V = self.fc_v(value)

    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

    energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

    if mask is not None:
      mask = mask[:, None, None, :]
      energy = energy.masked_fill(mask==0, -1e10)

    attention = torch.softmax(energy, dim=-1)
    x = torch.matmul(self.dropout(attention), V)
    x = x.permute(0, 2, 1, 3).contiguous()
    x = x.view(batch_size, -1, self.hid_dim)
    x = self.fc_o(x)

    return x, attention # возвращаем ещё attention, чтобы можно было при желании наблюдать за его изменением

Реализуем структуру нейронной сети с механизмом самовнимания

In [31]:
class GraphTransformer(nn.Module):
  def __init__(self, in_dim, max_seq_length, hid_dim=2048, dropout=0, device=device, n_heads=1):
    super(GraphTransformer, self).__init__()

    self.in_dim = in_dim
    self.device = device

    self.attn = MultiHeadAttentionLayer(hid_dim=self.in_dim,
                                        n_heads=n_heads,
                                        dropout=dropout,
                                        device=self.device) # Слой внимания

    self.ln1 = nn.LayerNorm(in_dim) # Нормализация всего слоя
    self.ln2 = nn.LayerNorm(in_dim)

    fc_blocks = []

    # fc_blocks.append(nn.Linear(self.in_dim, hid_dim).to(self.device))
    # fc_blocks.append(nn.Linear(hid_dim, self.in_dim).to(self.device))
    fc_blocks.append(nn.Linear(self.in_dim, 1).to(self.device))

    self.fc_blocks = fc_blocks # Полносвязный слой
    self.dropout = nn.Dropout(dropout) # Слой dropout, чтобы не застревать в локальном минимуме


  def forward(self, x, mask=None):
    attn_out1 = self.attn(x, x, x, mask=mask)[0]
    x = self.ln1(x + self.dropout(attn_out1))

    x = x[:, 0, :].squeeze().to(self.device) #size ([16], [25]) now
    # ff_out = self.fc_blocks[-2](F.relu(self.fc_blocks[-3](x)))
    x = self.ln2(x + self.dropout(ff_out))
    x = self.fc_blocks[-1](x)
    # print(x.shape)


    return x

Функция обучения модели и записи значений функции потерь на тренировочной и валидационной выборка, а также записи относительной ошибки

In [23]:
def fit(epochs, model, loss_func, opt, train_dl, val_dl):
  train_losses = []
  val_losses = []
  valid_re = []

  for epoch in range(epochs):

    # Переведём модель в состояние обучения
    model.train()
    loss_sum = 0
    for xb, yb in zip(*train_dl):
      #print(1)
      # Переведём всё на одно устройство
      xb, yb = xb.to(device), yb.to(device)

      loss = loss_func(model(xb).squeeze(), yb)
      loss_sum += loss.item()

      loss.backward()
      opt.step()
      opt.zero_grad()
    train_losses.append(loss_sum / len(train_dl[0]))

    # Будем иногда выводить значения функции потерь для мониторинга адекватности обучения
    if (epoch + 1) % 10 == 0:
      print(f'Epoch: {epoch + 1}')
      print(f'Loss: {train_losses[epoch]:.4f}')

    # Переведём модель в состояние предсказания
    model.eval()
    loss_sum = 0
    correct = 0
    num = 0
    with torch.no_grad():
      for xb, yb in zip(*val_dl):
        xb, yb = xb.to(device), yb.to(device)

        probs = model(xb).squeeze()
        loss_sum += loss_func(probs, yb).item()
        correct += (torch.abs(probs - yb)/yb).sum().item()
        num += len(xb)

    val_losses.append(loss_sum / len(val_dl[0]))
    valid_re.append(correct/num)

  return train_losses, val_losses, valid_re

Функция для предсказания

In [24]:
def predict(model, testset):
  model.eval()
  with torch.no_grad():
    preds = []
    for xb in testset[0]:
      pred = model(xb.to(device))
      preds.append(pred.squeeze())

  # Переведём preds в масств numpy для использования в метриках
  preds = np.array(torch.cat(preds[:-1]).cpu())
  y_test = []
  for y in testset[1]:
    y_test.append(y)

  # Переведём y_test в массив numpy для использования в метриках
  y_test = np.array(torch.cat(y_test[:-1]).numpy())

  return y_test, preds

In [29]:
IN_DIM = 25
HID_DIM = 100
EPOCHS = 60
BATCH_SIZE = 32
LR = 2e-3
MAX_SEQ_LENGTH = 141
HID_DIM = 512
N_HEADS = 1
DROPOUT = 0.5
LR = 1e-4

##Обучение модели для предсказания задержки


In [32]:
# Создание модели
model = GraphTransformer(in_dim=IN_DIM,
                          max_seq_length=MAX_SEQ_LENGTH,
                          hid_dim=HID_DIM,
                          n_heads=N_HEADS,
                          dropout=DROPOUT).to(device)

# Получение тензоров, разделённых на батчи
trainset, testset = get_dataloaders([X, y_delay], [X_test_d, y_test_d], BATCH_SIZE)

# Определение функции потерь и оптимизатора градиента
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))

# Запуск обучения и просмотр графика обучения
info = fit(epochs=50, model=model, loss_func=criterion, opt=optimizer, train_dl=trainset, val_dl=testset)
plot_training(*info)

y_test, y_pred = predict(model, testset)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
re = np.mean((np.abs(y_pred - y_test))/y_test)

print(f'MSE: {mse:.4f}, R²: {r2:.4f}, RE: {re:.4f}')

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

##Модель для предсказания площади


In [ ]:
IN_DIM = 25
HID_DIM = 100
EPOCHS = 60
BATCH_SIZE = 32
LR = 2e-3
MAX_SEQ_LENGTH = 141
HID_DIM = 256
N_HEADS = 1
DROPOUT = 0.5
LR = 1e-3

In [ ]:
# Создание модели
model = GraphTransformer(in_dim=IN_DIM,
                          max_seq_length=MAX_SEQ_LENGTH,
                          hid_dim=HID_DIM,
                          n_heads=N_HEADS,
                          dropout=DROPOUT).to(device)

# Получение тензоров, разделённых на батчи
trainset, testset = get_dataloaders([X, y_area], [X_test_a, y_test_a], BATCH_SIZE)

# Определение функции потерь и оптимизатора градиента
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))

# Обучение модели и просмотр графика обучения
info = fit(epochs=50, model=model, loss_func=criterion, opt=optimizer, train_dl=trainset, val_dl=testset)
plot_training(*info)

# Тестирование модели
y_test, y_pred = predict(model, testset)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
re = np.mean((np.abs(y_pred - y_test))/y_test)

print(f'MSE: {mse:.4f}, R²: {r2:.4f}, RE: {re:.4f}')